In [1]:
import pymeshlab
import numpy as np

In [2]:
ms = pymeshlab.MeshSet()
ms.load_new_mesh("model_normalized.obj")

m = ms.current_mesh()

v_matrix = m.vertex_matrix()
f_matrix = m.face_matrix()

In [3]:
def M_ijk_fast(f_matrix,v_matrix,max_m):
    num_faces = len(f_matrix)
    
    # Extract coordinates
    A = np.zeros([num_faces,3])
    B = np.zeros([num_faces,3])
    C = np.zeros([num_faces,3])
    for face_num in range(num_faces):
        [A[face_num,:],B[face_num,:],C[face_num,:]] = v_matrix[f_matrix[face_num]]
    # Calculate Determinants
    dets = np.linalg.det([[A[i],B[i],C[i]] for i in range(len(f_matrix))])
    
    # Allocate Tensors
    M_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    C_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    D_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    S_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    
    # Calculate C Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        C_tensor[:,i,j,k] = (C[:,0]**i)*(C[:,1]**j)*(C[:,2]**k)*(np.math.factorial(i+j+k)/(np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k)))
                        
    # Calculate D Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        if (i<0) or (j<0) or (k<0):
                            # D_ijk=0
                            pass
                        elif (0==i) and (0==j) and (0==k):
                            # D_ijk = 1
                            D_tensor[:,i,j,k] = 1
                        else:
                            D_tensor[:,i,j,k] = B[:,0]*D_tensor[:,i-1,j,k]+B[:,1]*D_tensor[:,i,j-1,k]+B[:,2]*D_tensor[:,i,j,k-1]+C_tensor[:,i,j,k]
    
    
    # Calculate S Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        if (i<0) or (j<0) or (k<0):
                            # S_ijk = 0
                            pass
                        elif (0==i) and (0==j) and (0==k):
                            # S_ijk = 1
                            S_tensor[:,i,j,k] = 1
                        else:
                            S_tensor[:,i,j,k] = A[:,0]*S_tensor[:,i-1,j,k]+A[:,1]*S_tensor[:,i,j-1,k]+A[:,2]*S_tensor[:,i,j,k-1]+D_tensor[:,i,j,k]
        
    # Calculate M Tensor, parallellized over faces
    for i in range(max_m):
        for j in range(max_m):
            for k in range(max_m):
                if (i+j+k)<=max_m:
                    M_tensor[:,i,j,k] = ((np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))/np.math.factorial(i+j+k+3))*dets[:]*S_tensor[:,i,j,k]
    return np.sum(M_tensor,axis=0)
#      M_tensor = M_tensor
#     return M_tensor

                        
    
    

In [4]:
def moment_loss_numpy(f_matrix_1, v_matrix_1, f_matrix_2, v_matrix_2, max_m = 10):
    return np.linalg.norm(M_ijk_fast(f_matrix_1, v_matrix_1,max_m)-M_ijk_fast(f_matrix_2, v_matrix_2,max_m))

In [5]:
moment_loss_numpy(f_matrix,v_matrix,f_matrix,v_matrix)

0.0